In [ ]:
import numpy as np
import pandas as pd

from bikewheelcalc.continuum_analysis import calc_buckling_tension
from doetools import TensionBucklingExpDOE
from wheel_library import wheel_from_name

from shutil import copy2
from imp import find_module

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
dyn_dir = '../data/doe/tension_buckling_dynamic'

up = 0.001e-3
doe = TensionBucklingExpDOE(out_dir=dyn_dir,
                            opts={'spk_paired': False,
                                  'spk_eltype': 'beam',
                                  'rim_perturb': [0., up, up, up, up],
                                  'sim_time': 1.5,
                                  'sim_temp_factor': 1.6})

w1 = wheel_from_name('std_elbows_in')
print(calc_buckling_tension(w1))

doe.add_experiment(w1, opts={'jobname': 'Tc_dynamic'})


w2 = wheel_from_name('vintage_road')

print(calc_buckling_tension(w2))

doe.add_experiment(w2, opts={'jobname': 'Tc_dynamic_lo_mu'})


print('\nCreated {0:d} simulations'.format(len(doe.db)))

doe.write_input_files(N_batches=1)
doe.to_csv()

# Copy postprocessing script
copy2(src=find_module('doetools')[1] + '/postproc_tension_exp.py',
      dst=dyn_dir);

In [ ]:
# Read result files
t = pd.read_csv(dyn_dir + '/Tc_dynamic_lo_mu_Pd.csv')
s = pd.read_csv(dyn_dir + '/Tc_dynamic_lo_mu_SE.csv')

# Calculate tightening strain
a_T = 4.484088e-03  # thermal "expansion" coefficient
t['Strain'] = -100.*a_T*t['Temperature']

# Interpolate strain energy from history data
t.loc[t['Step'] == 0, 'Strain Energy'] = np.interp(t.loc[t['Step'] == 0, 'Time [s]'],
                                                   s.loc[s['Step'] == 0, 'Time [s]'],
                                                   s.loc[s['Step'] == 0, 'Strain Energy [J]'])

t.loc[t['Step'] == 1, 'Strain Energy'] = np.interp(t.loc[t['Step'] == 1, 'Time [s]'],
                                                   s.loc[s['Step'] == 1, 'Time [s]'],
                                                   s.loc[s['Step'] == 1, 'Strain Energy [J]'])

fig, ax = plt.subplots(ncols=2, figsize=(6.5, 3.))

cm = sns.color_palette('Blues', 4)

ax[0].plot(t.loc[t['Step'] == 1, 'Strain'],
           t.loc[t['Step'] == 1, 'Tension [N]']/1000, color=cm[1])
ax[0].plot(t.loc[t['Step'] == 0, 'Strain'],
           t.loc[t['Step'] == 0, 'Tension [N]']/1000, color=cm[2])

ax[0].set_xlim([0., 0.8])
ax[0].set_xlabel('Tightening strain [%]')

ax[0].set_ylim([0., 2.])
ax[0].set_ylabel('Average tension [kN]')


# STRAIN ENERGY
ax[1].plot(-100.*a_T*t.loc[t['Step'] == 1, 'Temperature'],
           t.loc[t['Step'] == 1, 'Strain Energy'], color=cm[1])
ax[1].plot(-100.*a_T*t.loc[t['Step'] == 0, 'Temperature'],
           t.loc[t['Step'] == 0, 'Strain Energy'], color=cm[2])

# Extrapolate quadratic strain energy
i_max = int(1.0*t['Tension [N]'].idxmax()) - 1
temp_crit = t.iloc[t['Tension [N]'].idxmax()]['Temperature']
p_2 = np.polyfit(t.iloc[:i_max]['Temperature'], t.iloc[:i_max]['Strain Energy'], 2)

tt = np.linspace(temp_crit, 2.0*temp_crit, 100)
ax[1].plot(-100.*a_T*tt, np.polyval(p_2, tt), '--', color=cm[2])

# Mark bifurcation point
ax[1].plot(-100.*a_T*t.loc[t['Tension [N]'].idxmax(), 'Temperature'],
           t.loc[t['Tension [N]'].idxmax(), 'Strain Energy'], 'C3*')

ax[1].set_xlim([0., 0.8])
ax[1].set_xlabel('Tightening strain [%]')

ax[1].set_ylim([0., 80])
ax[1].set_ylabel('Strain energy [J]');


plt.tight_layout()
plt.savefig('../figs/buckling_tension/_python_tension_buckling_dynamic.pdf')